In [1]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    count = 0
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        print("loop: ", i, "sup: ", sup, "length: ", len(sup))
        if len(sup) == 11:
            count += 1
            print("count: ",count)
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup  # return shortest

In [2]:
scs(['CCT', 'CTT','TGC', 'TGG', 'GAT', 'ATT'])


loop:  4 sup:  CCTTGCTGGATT length:  12
loop:  4 sup:  CCTTGCTGGATTGAT length:  15
loop:  4 sup:  CCTTGCGATGGATT length:  14
loop:  4 sup:  CCTTGCGATTGG length:  12
loop:  4 sup:  CCTTGCATTGGAT length:  13
loop:  4 sup:  CCTTGCATTGATGG length:  14
loop:  4 sup:  CCTTGGTGCGATT length:  13
loop:  4 sup:  CCTTGGTGCATTGAT length:  15
loop:  4 sup:  CCTTGGATGCATT length:  13
loop:  4 sup:  CCTTGGATTGC length:  11
count:  1
loop:  4 sup:  CCTTGGATTGCGAT length:  14
loop:  4 sup:  CCTTGGATTGATGC length:  14
loop:  4 sup:  CCTTGATGCTGGATT length:  15
loop:  4 sup:  CCTTGATGCATTGG length:  14
loop:  4 sup:  CCTTGATGGTGCATT length:  15
loop:  4 sup:  CCTTGATGGATTGC length:  14
loop:  4 sup:  CCTTGATTGCTGG length:  13
loop:  4 sup:  CCTTGATTGGTGC length:  13
loop:  4 sup:  CCTTATTGCTGGAT length:  14
loop:  4 sup:  CCTTATTGCGATGG length:  14
loop:  4 sup:  CCTTATTGGTGCGAT length:  15
loop:  4 sup:  CCTTATTGGATGC length:  13
loop:  4 sup:  CCTTATTGATGCTGG length:  15
loop:  4 sup:  CCTTATTGATGGTGC 

'CCTTGGATTGC'

In [3]:
len('CCTTGGATTGC')

11

In [34]:
def overlap(a, b, min_length):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match


In [35]:
#Because the reads are quite large. len(seqs) = 1340. 
#Our greedy algorithm code is quite slow as it has to do overlap comparsions a total of :
#n! + n-1! + n-2! ... 1 times, which will be very very large
#So lets accelerate the algorithm.

def pick_maximal_overlap(reads, k):
    reada, readb = None, None
    best_olen = 0
    greedy = {}
    for a,b in itertools.permutations(reads, 2):
        if not greedy.get((a,b),None) == None: 
    #searches the keys in the greedy dictionary and returns value, else returns None
            continue
        olen = overlap(a, b, min_length=k)
        greedy[(a,b)] = olen
        if olen > best_olen:
            reada, readb = a, b
            best_olen = olen
    return reada, readb, best_olen

def greedy_scs(reads, k):
    read_a, read_b, olen = pick_maximal_overlap(reads, k)
    while olen > 0:
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        read_a, read_b, olen = pick_maximal_overlap(reads, k)
    return ''.join(reads) #even though there are no more overlaps/edges in the overlap graph but there still are reads/nodes left we would concatinate them as well.


In [36]:
def readFastq(filename):
    sequences = []
    qualities = []
    n = 0
    l = []
    with open(filename, 'r') as fh:
        while True:
            fh.readline() #skip name line
            seq = fh.readline().rstrip() #read base sequences
            fh.readline() #skip placeholder line
            qual = fh.readline().rstrip() # read base quality line
            if len(seq) == 0:
                break #end of the file
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities
seqs, _ = readFastq("ads1_week4_reads.fastq")


In [37]:
len(seqs)


1881

In [ ]:
genome = greedy_scs(seqs, 10)
#This is still very slow.

In [15]:
# The virus genome should be 15,894 bases long.
len(genome)

188100

In [25]:
#Because the reads are quite large. len(seqs) = 1340. 
#Our greedy algorithm code is quite slow as it has to do overlap comparsions a total of :
#n! + n-1! + n-2! ... 1 times, which will be very very large
#So lets accelerate the algorithm.

#This could be done by making a index of all the text/reads where we store the prefix of k length and 
#associate the prefix with a set of reads that contain it. 
#'ACCATT': set(ACCATTGGGGGG, ACCATTAAAAA)
#Then querry the suffix of each pattern/reads in the index, find the associated read in the set
#If the read does not match the read that suffix belongs to, then do overlap comparsions
#This ensures only performing overlap comparisions between reads that actually has suffix=prefix.

#reads are the reads in fastq files and k is the minimum overlap required
def pick_maximal_overlap_index(reads, k):
    index = {} #making an index with the help of dictionary 
    for read in reads:
        kmers = [] #to store a list of prefix of length 'k'
        for i in range(len(read) - k + 1): #generating prefixes from text/reads
            kmers.append(read[i:i+k])
        for kmer in kmers: #Then storing the k-mer prefixes in the dictionary
            if kmer not in index:
                index[kmer] = set() #value is stored in set so as to only store unique reads
            index[kmer].add(read) #the reads are stored as the values and their prefixes are the keys
            
    #Now storing the reads with maximum edges/olen for greedy algorithm
    reada, readb = None, None
    best_olen = 0
    #Only finding overlap whoes read1_suffix == read2_prefix
    for a in reads: 
        #Compare the suffix of the pattern/reads in the index 
        for b in index[a[-k:]]: #a[-k:] is the suffix of length 'k' (last k characters of the string 'a')
            if a != b: #b is the read associated with the prefix who matches the suffix
                olen = overlap(a, b, k)
                if olen > best_olen:
                    reada, readb = a, b
                    best_olen = olen
    return reada, readb, best_olen


#This function implements the greedy shortest common superstring algorithm using an accelerated version of pick_maximal_overlap(reads, k).

def greedy_scs_index(reads, k):
    read_a, read_b, olen = pick_maximal_overlap_index(reads, k)
    while olen > 0:
        #print(len(reads))
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        read_a, read_b, olen = pick_maximal_overlap_index(reads, k)
    return ''.join(reads)



In [26]:
genome = greedy_scs_index(seqs, 10)

In [27]:
# The virus genome should be 15,894 bases long.
len(genome)

15894

In [28]:
genome.count('A')

4633

In [29]:
genome.count('T')

3723

In [ ]:
#Using algorithm for total number of A's and T's in the assembled genome
def findBases(genome, b):
    a = 0
    for base in genome:
        if base == b:
            a += 1
    return a
# Counting the number of As in the fully assembled genome:
print(findBases(genome, 'A'))

print(findBases(genome, 'T'))
